## importando o dataset

In [1]:
import pandas as pd
train = pd.read_csv('winequality-white.csv')

In [2]:
train.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


### Altera a coluna "quality" transformando-a em binaria. Se a nota eh maior ou igual a 7, "quality" recebe 1, senao recebe 0. Pular este passo para testar o resultado com multiplos valores para a coluna "quality"

In [3]:
for dataset in [train]:
    dataset.loc[(dataset.quality < 7), 'quality'] = 0
    dataset.loc[(dataset.quality >= 7), 'quality'] = 1
    

### histograma da coluna "quality"

In [4]:
# print train[(train['quality'>=8])]
print len(train[train.quality == 1])
train['quality'].plot.hist()

1060


In [5]:
import numpy as np
from collections import Counter
from sklearn.cross_validation import cross_val_score

X_df = train[['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'chlorides', 'pH', 'sulphates', 'alcohol']]
Y_df = train['quality']
#Xdummies_df = pd.get_dummies(X_df)
# X = Xdummies_df.values
X = X_df.values
Y = Y_df.values

porcentagem_de_treino = 0.9

tamanho_de_treino = int(porcentagem_de_treino * len(Y))

treino_dados = X[0:tamanho_de_treino]
treino_marcacoes = Y[0:tamanho_de_treino]

validacao_dados = X[tamanho_de_treino:]
validacao_marcacoes = Y[tamanho_de_treino:]

result_teste = []

def fit_and_predict_kf(nome, modelo, dados, marcacoes):
    k = 10
    scores = cross_val_score(modelo, dados, marcacoes, cv = k)
    
    taxa_de_acerto = np.mean(scores)
    result_teste.append({nome: taxa_de_acerto})

    print("Acerto %s: %.2f" % (nome, taxa_de_acerto*100))
    
result_validacao = []


/home/asus/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
from sklearn.naive_bayes import GaussianNB
modeloGaussian = GaussianNB()
fit_and_predict_kf("GaussianNB", modeloGaussian, treino_dados, treino_marcacoes)

Acerto GaussianNB: 73.61


In [7]:
from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
fit_and_predict_kf("kf MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)

Acerto kf MultinomialNB: 73.75


In [8]:
from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
fit_and_predict_kf("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)

Acerto AdaBoostClassifier: 79.67


In [9]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0))
fit_and_predict_kf("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)

Acerto OneVsRest: 58.57


In [10]:
from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0))
fit_and_predict_kf("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)

Acerto OneVsOne: 58.57


In [11]:
from sklearn.ensemble import RandomForestClassifier
modeloRandomForest = RandomForestClassifier()
fit_and_predict_kf("RandomForest", modeloRandomForest, treino_dados, treino_marcacoes)

Acerto RandomForest: 79.99


In [12]:
print(result_teste)

[{'GaussianNB': 0.73609751359751363}, {'kf MultinomialNB': 0.73748688752890423}, {'AdaBoostClassifier': 0.79671285297335714}, {'OneVsRest': 0.58573466449516876}, {'OneVsOne': 0.58573466449516876}, {'RandomForest': 0.79989775164144916}]
